In [1]:
! pip install langgraph

In [2]:
! pip install langchain langchain_core langchain_groq langchain_community langchain

In [3]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import Image, display

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define Agent

In [4]:
class PlannerState(TypedDict):
  messages : Annotated[List[AIMessage | HumanMessage], " the messages in the conversation"]
  city: str
  interest: List[str]
  itinerary: str

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()  # load variables from .env file

API_KEY = os.getenv("GROQ_API_KEY")


In [6]:
from ast import mod
from langchain_groq import ChatGroq
llm= ChatGroq(
    temperature=0,
    groq_api_key=API_KEY,
    model_name="llama-3.3-70b-versatile",
)
result= llm.invoke(" what is multi AI agent")
result.content

'A multi-agent system is a system that consists of multiple intelligent agents that interact with each other and their environment to achieve a common goal or set of goals. In the context of artificial intelligence (AI), a multi-agent system is a system that comprises multiple AI agents, each with its own capabilities, goals, and behaviors, that work together to achieve a shared objective.\n\nIn a multi-agent system, each agent can be thought of as a separate entity that perceives its environment, makes decisions, and takes actions to achieve its goals. The agents can be homogeneous (i.e., identical) or heterogeneous (i.e., different), and they can interact with each other in various ways, such as:\n\n1. **Cooperation**: Agents work together to achieve a common goal.\n2. **Competition**: Agents compete with each other to achieve individual goals.\n3. **Coordination**: Agents coordinate their actions to achieve a shared goal.\n\nMulti-agent systems can be used to model and solve complex

In [7]:
itinerary_prompt=ChatPromptTemplate.from_messages([
    ("system", " you are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's {interest}. Provide a brief, bulleted itinerary. "),
    ("human", "Create an itinerary for my day trip.")
])

## Define agent function

In [8]:
def input_city( state: PlannerState) -> PlannerState:
  print(" Enter the city you want to visit for your day trip: ")
  user_message= input(" Your Input: ")
  return{
      **state,
      "city": user_message,
      "messages": state["messages"] + [HumanMessage(content=user_message)]
  }


In [9]:
def input_interest( state: PlannerState) -> PlannerState:
  print(f" Enter your interest for the trip:{ state['city']}( comma-seperated) ")
  user_message= input(" Your Input: ")
  return{
      **state,
      "interest": [interest.strip() for interest in user_message.split(",")],
      "messages": state["messages"] + [HumanMessage(content=user_message)]
  }


In [10]:
def create_itinerary( state: PlannerState) -> PlannerState:
  print(f" Create an itinerary for { state['city']} based on interest: {','.join(state['interest'])}"),
  response= llm.invoke(itinerary_prompt.format_prompt(city=state["city"], interest=",".join(state["interest"])))
  print("\n Final Itinerary: ")
  print(response.content)
  return{
      **state,
      "messages": state["messages"] + [AIMessage(content=response.content)],
      "itinerary": response.content
  }


## Cretae and Compile the Graph

In [11]:
workflow=StateGraph(PlannerState)
workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("create_itinerary", create_itinerary)



In [12]:
workflow.set_entry_point("input_city")
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "create_itinerary")
workflow.add_edge("create_itinerary", END)


In [13]:
app=workflow.compile()

## Display the graph structure

In [29]:
# display(
#     Image(
#         app.get_graph().draw_mermaid_png(draw_method= MermaidDrawMethod.PYPPETEER)
#     )
# )

## Define functions that runs the graph

In [15]:
def travel_planner(user_request: str):
  print(f"initial Request: { user_request}\n")
  state={
      "messages": [HumanMessage(content=user_request)],
      "city": "",
      "interest": [],
      "itinerary": ""
  }
  for output in app.stream(state):
    pass


In [18]:
user_request= " Plan a day trip"
travel_planner(user_request)

initial Request:  Plan a day trip

 Enter the city you want to visit for your day trip: 
 Enter your interest for the trip:guntur( comma-seperated) 
 Create an itinerary for guntur based on interest: temple,island,vijayawada

 Final Itinerary: 
Here's a brief, bulleted itinerary for a day trip from Guntur based on your interests:

* 8:00 AM: Start the day with a visit to the **Amaravathi Temple**, a historic temple dedicated to Lord Shiva.
* 10:00 AM: Head to **Bhavani Island**, a scenic island on the Krishna River, perfect for a relaxing stroll or a picnic.
* 1:00 PM: Take a break for lunch at a local restaurant in **Vijayawada**, trying some of the city's famous cuisine.
* 2:30 PM: Explore the **Kanakadurga Temple** in Vijayawada, a famous temple dedicated to Goddess Durga.
* 5:00 PM: End the day with a visit to the **Prakasam Barrage** in Vijayawada, a scenic spot with great views of the Krishna River.

This itinerary should give you a good mix of temple visits, island relaxation, a

### create a simple front end app

In [19]:
!pip install gradio

  Using cached websockets-15.0.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached hf_xet-1.1.10-cp37-abi3-macosx_11_0_arm64.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 52.9 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 25.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 51.2 MB/s  0:00:00
Using cached websockets-15.0.1-cp312-cp312-macosx_11_0_arm64.whl (173 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.7/815.7 kB 34.9 MB/s  0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.29.1
    Uninstalling huggingface-hub-0.29.1:
      Successfully uninstalled huggingface-hub-0.29.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [gradio]10/11 [gradio]face-hub]
ERROR:

In [20]:
import gradio as gr

In [21]:
def input_city1( city: str, state: PlannerState) -> PlannerState:
  return{
      **state,
      "city":city,
      "messages": state["messages"] + [HumanMessage(content=city)]
  }


In [22]:
def input_interest1( interest: str, state: PlannerState) -> PlannerState:

  return{
      **state,
      "interest": [interest.strip() for interest in interest.split(",")],
      "messages": state["messages"] + [HumanMessage(content=interest)]
  }


In [23]:
def create_itinerary1( state: PlannerState) -> str:
  response= llm.invoke(itinerary_prompt.format_messages(city=state["city"], interest=",".join(state["interest"])))
  state['itinerary']= response.content
  state['messages']+=[AIMessage(content=response.content)]
  return response.content


In [24]:
def travel_planner1(city:str, interest:str):
  state={
      "messages":[],
      "city": "",
      "interest":[],
      "itinerary": "",
  }
  state=input_city1(city,state)
  state=input_interest1(interest, state)
  itinerary=create_itinerary1(state)
  return itinerary

In [25]:
interface= gr.Interface(
    fn=travel_planner1,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="Enter the city name"),
        gr.Textbox(label="Enter your interests(comma-seperated)")
    ],
    outputs=[
        gr.Textbox(label="Generated Itinerary", lines=20, show_copy_button=True)
    ],
    title=" Travel Itinerary Planner",
    description=" Enter city and your interests to generate a personalized day trip itinerary"
)
interface.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
